In [ ]:
### Librerias

import os
import pandas as pd
import networkx as nx
from tqdm import tqdm
import imageio.v2 as imageio
import matplotlib.pyplot as plt
from collections import defaultdict, deque

In [ ]:
##> Función de carga y orden para grafos

def cargar_y_ordenar_grafos(ruta_base):
    """
    Carga archivos .graphml desde subcarpetas de una ruta base y los ordena cronológicamente.

    Parámetros:
    -----------
    ruta_base : str
        Ruta principal donde se encuentran carpetas que contienen archivos .graphml.
        Cada carpeta debe tener como nombre un periodo en formato 'MesAño' (ej. 'Marzo2021').

    Retorna:
    --------
    grafo_dict_ordenado : dict
        Diccionario donde:
        - Las claves son los nombres de las carpetas (periodos).
        - Los valores son grafos cargados con NetworkX desde archivos .graphml.
        - El diccionario está ordenado cronológicamente por trimestre (Marzo, Junio, Septiembre, Diciembre) y año.

    Proceso:
    --------
    - Busca recursivamente todos los archivos .graphml dentro de las subcarpetas de `ruta_base`.
    - Asocia cada archivo con el nombre de su carpeta contenedora como clave.
    - Ordena los grafos con base en un orden trimestral personalizado.
    """
    grafo_temp = {}

    for root, dirs, files in os.walk(ruta_base):
        for file in files:
            if file.endswith(".graphml"):
                carpeta = os.path.basename(root)
                ruta_completa = os.path.join(root, file)
                G = nx.read_graphml(ruta_completa)
                grafo_temp[carpeta] = G

    print(f"Se cargaron {len(grafo_temp)} grafos.")

    orden_meses = ['Marzo', 'Junio', 'Septiembre', 'Diciembre']

    def clave_orden(periodo):
        for mes in orden_meses:
            if periodo.startswith(mes):
                anio = int(periodo.replace(mes, ''))
                return (anio, orden_meses.index(mes))
        return (9999, 99)  # Fallback para casos no reconocidos

    grafo_dict_ordenado = dict(sorted(grafo_temp.items(), key=lambda x: clave_orden(x[0])))
    return grafo_dict_ordenado


In [ ]:
### Ejecución funcion de carga y orden para grafos

Salida = 'D:/Users/LAAR8976/Desktop/LADINO_RED/NEW_PARVADA2/'

grafos_ordenados = cargar_y_ordenar_grafos(Salida)

print("\n Periodos disponibles:", list(grafos_ordenados.keys()))

In [ ]:
##> Función de rastreo grafos
def rastrear_en_grafos_graphml(grafos_ordenados, id_empleado):
    """
    Rastrea la presencia y posición jerárquica de un empleado a lo largo de una serie de grafos ordenados temporalmente.

    Parámetros:
    -----------
    grafos_ordenados : dict
        Diccionario de grafos en formato NetworkX, ordenados por periodo (clave).
        Las claves son cadenas representando los periodos (ej. 'Marzo2021').
        Los valores son grafos dirigidos cargados desde archivos .graphml.

    id_empleado : int o str
        Identificador del empleado a rastrear. Se convierte a string para su búsqueda en los nodos del grafo.

    Retorna:
    --------
    resultados : list of dict
        Lista de diccionarios con la información del nodo en cada periodo en que aparece. Cada entrada contiene:
        - "periodo": nombre del periodo.
        - "jefe_directo": ID del nodo superior inmediato, o None si no tiene.
        - "subordinados": lista de IDs de nodos subordinados.
        - "titulo_puesto": nombre del puesto del empleado (atributo del nodo).
        - "nivel_direccion": nivel jerárquico (atributo del nodo).
        - "admon_gral": administración general a la que pertenece el nodo (atributo del nodo).

    Notas:
    ------
    - Si el empleado no se encuentra en un periodo, ese periodo se omite del resultado.
    - Se espera que los grafos sean dirigidos (DiGraph) y contengan metadatos por nodo.
    """
    resultados = []

    for periodo, G in grafos_ordenados.items():
        nodo_id = str(id_empleado)

        if nodo_id in G.nodes:
            jefe_directo = list(G.predecessors(nodo_id))
            subordinados = list(G.successors(nodo_id))

            info = {
                "periodo": periodo,
                "jefe_directo": jefe_directo[0] if jefe_directo else None,
                "subordinados": subordinados,
                "titulo_puesto": G.nodes[nodo_id].get("titulo_puesto", "Sin info"),
                "nivel_direccion": G.nodes[nodo_id].get("nivel_direccion", "Sin info"),
                "admon_gral": G.nodes[nodo_id].get("admon_gral", "Sin info")
            }
            resultados.append(info)

    return resultados


In [ ]:
resultados = rastrear_en_grafos_graphml(grafos_ordenados, "31640")

for r in resultados:
    print(f"\n Periodo: {r['periodo']}")
    print(f" Título: {r['titulo_puesto']} | Nivel: {r['nivel_direccion']} | Admon: {r['admon_gral']}")
    
    print(" Jefe directo:")
    print(f"   - {r['jefe_directo']}" if r['jefe_directo'] else "   Ninguno")

    print(" Subordinados:")
    if r['subordinados']:
        for sub in r['subordinados']:
            print(f"   - {sub}")
    else:
        print("   Ninguno")


In [ ]:
##> Validación de la existencia del nodo
def validar_existencia_nodo(grafos, nodo_id):
    """
    Verifica la existencia de un nodo específico en una serie de grafos por periodo y muestra los resultados.

    Parámetros:
    -----------
    grafos : dict
        Diccionario de grafos (por ejemplo, generados con NetworkX), donde:
        - Las claves son nombres de periodos (str).
        - Los valores son objetos de grafo (NetworkX DiGraph o Graph).

    nodo_id : int o str
        Identificador del nodo a verificar. Se convierte a cadena para la búsqueda en los grafos.

    Retorna:
    --------
    None
        Esta función imprime directamente el resultado de la verificación por cada periodo.
        No retorna un valor explícito, pero informa visualmente en consola si el nodo está presente o no en cada grafo.

    Ejemplo de salida:
    ------------------
    Verificación de existencia del nodo:
     - Marzo2021:  Presente
     - Junio2021:  NO encontrado
     ...
    """
    nodo_str = str(nodo_id)
    print("\n Verificación de existencia del nodo:")
    for periodo, G in grafos.items():
        existe = nodo_str in G.nodes
        print(f" - {periodo}: {' Presente' if existe else ' NO encontrado'}")


In [ ]:
def validar_estructura_nodo(grafos, nodo_id):
    """
    Verifica la estructura jerárquica de un nodo específico en una serie de grafos por periodo.
    
    Para cada periodo:
    - Informa si el nodo está presente.
    - Lista sus jefes directos (predecesores).
    - Lista sus subordinados directos (sucesores).

    Parámetros:
    -----------
    grafos : dict
        Diccionario de grafos dirigidos (por ejemplo, NetworkX DiGraph), donde:
        - Las claves son cadenas con los nombres de los periodos (ej. "Marzo2021").
        - Los valores son objetos grafo dirigidos.

    nodo_id : int o str
        Identificador del nodo a analizar. Se convierte a cadena para su búsqueda en los grafos.

    Retorna:
    --------
    None
        La función imprime directamente en consola la información de estructura jerárquica del nodo
        para cada periodo. No devuelve ningún valor explícito.

    Ejemplo de salida:
    ------------------
     Análisis estructural del nodo 12345:
     - Marzo2021: Presente
        ↳ Jefes directos      : ['9876']
        ↳ Subordinados directos: ['5678', '6789']
     - Junio2021:  NO encontrado
    """
    nodo_str = str(nodo_id)
    print(f"\n Análisis estructural del nodo {nodo_str}:")

    for periodo, G in grafos.items():
        if nodo_str in G.nodes:
            jefes = list(G.predecessors(nodo_str))
            subordinados = list(G.successors(nodo_str))

            print(f" - {periodo}: Presente")
            print(f"    ↳ Jefes directos      : {jefes if jefes else 'Ninguno'}")
            print(f"    ↳ Subordinados directos: {subordinados if subordinados else 'Ninguno'}")
        else:
            print(f" - {periodo}: NO encontrado")


In [ ]:
validar_estructura_nodo(grafos_ordenados, "31640") 

In [ ]:
##> Funcion para ordenar periodos claves
def ordenar_periodos_claves(claves):
    """
    Ordena una lista de claves que representan periodos en formato 'analisis_MesAño' según un orden trimestral definido.

    Parámetros:
    -----------
    claves : list of str
        Lista de cadenas con nombres de periodos con prefijo 'analisis_', seguidos de un mes y un año (ej. 'analisis_Marzo2021').

    Retorna:
    --------
    list of str
        Lista de claves ordenadas cronológicamente con base en el orden trimestral: 
        ['Marzo', 'Junio', 'Septiembre', 'Diciembre'] y luego por año.

    Ejemplo:
    --------
    Entrada: ['analisis_Junio2022', 'analisis_Marzo2021', 'analisis_Diciembre2021']
    Salida : ['analisis_Marzo2021', 'analisis_Diciembre2021', 'analisis_Junio2022']

    Notas:
    ------
    - Las claves mal formateadas o no reconocidas serán colocadas al final.
    - El orden se basa en tuplas (año, índice_mes) para garantizar orden cronológico trimestral.
    """
    orden_meses = ['Marzo', 'Junio', 'Septiembre', 'Diciembre']

    def clave_orden(clave):
        periodo = clave.replace("analisis_", "")
        for mes in orden_meses:
            if periodo.startswith(mes):
                anio = int(periodo.replace(mes, ''))
                return (anio, orden_meses.index(mes))
        return (9999, 99)  # Para claves no reconocidas

    return sorted(claves, key=clave_orden)


In [ ]:
##> Funcion para la tabla jerarquica
def construir_tabla_jerarquica(grafos_ordenados):
    """
    Construye una tabla jerárquica a partir de una serie de grafos organizacionales.

    Parámetros:
    -----------
    grafos_ordenados : dict
        Diccionario de grafos dirigidos (NetworkX DiGraph) ordenados cronológicamente.
        Las claves representan periodos (ej. 'Marzo2021') y los valores son grafos con metadatos por nodo,
        incluyendo el atributo 'nivel_direccion'.

    Retorna:
    --------
    df : pandas.DataFrame
        DataFrame donde:
        - Las filas corresponden a los nodos (id_empleado).
        - Las columnas son los distintos periodos.
        - Cada celda contiene el nivel jerárquico (`nivel_direccion`) del nodo en ese periodo.
        - Si el nodo no aparece o el valor es inválido, se asigna `None`.

    Detalles del procesamiento:
    ---------------------------
    - Extrae el atributo `nivel_direccion` de cada nodo.
    - Intenta convertir los niveles a enteros para asegurar la comparabilidad.
    - Ordena cronológicamente las columnas usando la función `ordenar_periodos_claves`.
    - La tabla resultante permite hacer análisis de evolución jerárquica por nodo en el tiempo.

    Requiere:
    ---------
    - `pandas` importado como `pd`.
    - La función auxiliar `ordenar_periodos_claves` para ordenar los periodos.
    """
    data = {}

    for periodo, G in grafos_ordenados.items():
        niveles = {}
        for nodo, attrs in G.nodes(data=True):
            nivel = attrs.get("nivel_direccion")
            if nivel is not None:
                try:
                    niveles[nodo] = int(nivel)
                except:
                    niveles[nodo] = None
        data[periodo] = niveles

    df = pd.DataFrame.from_dict(data, orient="columns")
    df.index.name = "id_empleado"
    df = df.sort_index(axis=1, key=lambda x: ordenar_periodos_claves(x))  # Ordenar columnas

    return df


In [ ]:
##> Función de los cambios
def clasificar_cambios_jerarquicos(serie):
    """
    Clasifica los cambios jerárquicos de un nodo a lo largo del tiempo con base en su nivel_direccion.

    Parámetros:
    -----------
    serie : pandas.Series
        Serie de niveles jerárquicos (`nivel_direccion`) de un nodo, indexada por periodos.
        Se espera que contenga valores numéricos (enteros) o NaN (cuando el nodo no está presente en un periodo).

    Retorna:
    --------
    resultado : list of str
        Lista de etiquetas de cambio por periodo, con la misma longitud que la serie.
        Cada posición representa el cambio respecto al valor anterior:
        - "Inicio": primer valor de la serie.
        - "Igual": no hubo cambio de nivel.
        - "Subió": el valor del nivel disminuyó (ascenso jerárquico).
        - "Bajó": el valor del nivel aumentó (descenso jerárquico).
        - "Reapareció": el nodo estaba ausente (NaN) y vuelve a aparecer.
        - "Desapareció": el nodo estaba presente y desaparece en el periodo siguiente.
        - "Sin cambio detectable": cualquier otro caso no identificado explícitamente.

    Ejemplo:
    --------
    Entrada: [3, 3, 2, NaN, 2, 2, NaN, 1]
    Salida : ['Inicio', 'Igual', 'Subió', 'Desapareció', 'Reapareció', 'Igual', 'Desapareció', 'Reapareció']
    """
    resultado = []
    valores = serie.values

    for i in range(len(valores)):
        curr = valores[i]
        if i == 0:
            resultado.append("Inicio")
        else:
            prev = valores[i - 1]
            if pd.isna(prev) and not pd.isna(curr):
                resultado.append("Reapareció")
            elif not pd.isna(prev) and pd.isna(curr):
                resultado.append("Desapareció")
            elif prev == curr:
                resultado.append("Igual")
            elif curr < prev:
                resultado.append("Subió")
            elif curr > prev:
                resultado.append("Bajó")
            else:
                resultado.append("Sin cambio detectable")
    return resultado


In [ ]:
# Construir la tabla de niveles
## > restructurar para que sea una función
df_niveles = construir_tabla_jerarquica(grafos_ordenados)

# Extraer la serie del nodo de interés
nodo_objetivo = "31640"
serie_ = df_niveles.loc[nodo_objetivo]

# Clasificar los cambios
cambios_ = clasificar_cambios_jerarquicos(serie_)

# Crear DataFrame resumen
df_j = pd.DataFrame({"Periodo": serie_.index,
                     "Nivel": serie_.values,
                     "Cambio": cambios_})

# Reordenar los periodos como categoría ordenada
df_j["Periodo"] = pd.Categorical(df_j["Periodo"],
                                 categories=ordenar_periodos_claves(df_j["Periodo"].unique()),
                                 ordered=True)

# Ordenar el DataFrame
df_j = df_j.sort_values("Periodo")

# Mostrar resultados
print(" Evolución jerárquica del nodo:", nodo_objetivo)
display(df_j)